## Combine the daily datasets
- ### [Kaggle 1990 to 2020 dataset](https://www.kaggle.com/stoicstatic/india-stock-data-nse-1990-2020)
- ### [Kaggle 2020 half+ to 2021 Q1 + Q2(half)](https://www.kaggle.com/unknovvnuser/nseindia-all-stocks)

### We'll drop them into individual files where file names corrospond to Stock Symbol/Index Symbol

### Once the file data is consolidates, we'll also use AlphaVantage/NSE or some api to fill the missing latest data

In [1]:
import os
import pandas as pd
import dateutil
from bs_threading import bs_threadify

In [3]:
for dirname, subdir, filename in os.walk("input"):
    print(dirname)


for dirname, subdir, filenames in os.walk("input\\kaggle nseindia-all-stocks"):
    print(filenames)
    
for dirname, subdir, filenames in os.walk("input\\kaggle daily NSE data\\SCRIPT"):
    print(filenames)

input
input\all_hist_data
input\kaggle daily NSE data
input\kaggle daily NSE data\INDEX
input\kaggle daily NSE data\SCRIPT
input\kaggle nseindia-all-stocks
['archive.zip', 'combined.csv']
['20MICRONS.csv', '21STCENMGM.csv', '3IINFOTECH.csv', '3MINDIA.csv', '3PLAND.csv', '5PAISA.csv', '63MOONS.csv', '8KMILES.csv', 'A2ZINFRA.csv', 'AAKASH.csv', 'AARON.csv', 'AARTIDRUGS.csv', 'AARTIIND.csv', 'AARTISURF.csv', 'AARVEEDEN.csv', 'AARVI.csv', 'AAVAS.csv', 'ABAN.csv', 'ABB.csv', 'ABBOTINDIA.csv', 'ABCAPITAL.csv', 'ABFRL.csv', 'ABMINTLTD.csv', 'ACC.csv', 'ACCELYA.csv', 'ACE.csv', 'ADANIENT.csv', 'ADANIGAS.csv', 'ADANIGREEN.csv', 'ADANIPORTS.csv', 'ADANIPOWER.csv', 'ADANITRANS.csv', 'ADFFOODS.csv', 'ADHUNIKIND.csv', 'ADL.csv', 'ADORWELD.csv', 'ADROITINFO.csv', 'ADSL.csv', 'ADVANIHOTR.csv', 'ADVENZYMES.csv', 'AEGISCHEM.csv', 'AFFLE.csv', 'AGARIND.csv', 'AGCNET.csv', 'AGRITECH.csv', 'AGROPHOS.csv', 'AHLEAST.csv', 'AHLUCONT.csv', 'AHLWEST.csv', 'AIAENG.csv', 'AIRAN.csv', 'AJANTPHARM.csv', 'AJMERA.cs

In [4]:
sample_scripts_df = pd.read_csv("input\\kaggle daily NSE data\\SCRIPT\\20MICRONS.csv")

In [5]:
sample_combined_df = pd.read_csv("input\\kaggle nseindia-all-stocks\\combined.csv")

In [6]:
print(sample_scripts_df.columns.values)
print(sample_combined_df.columns.values)

['Date' 'Symbol' 'Series' 'Prev Close' 'Open' 'High' 'Low' 'Last' 'Close'
 'VWAP' 'Volume' 'Turnover' 'Trades' 'Deliverable Volume' '%Deliverble']
['Symbol' 'Series' 'Date' 'Prev Close' 'Open Price' 'High Price'
 'Low Price' 'Last Price' 'Close Price' 'Average Price'
 'Total Traded Quantity' 'Turnover' 'No. of Trades' 'Deliverable Qty'
 '% Dly Qt to Traded Qty']


In [7]:
# The date in this one don't match that of NSE data we ought to convert it
sample_scripts_df.sample()
_month_index = ["","Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
def convert_date_to_nse_format(scriptdate):
    parts_old = scriptdate.split("-")
    parts_new = [parts_old[2], _month_index[int(parts_old[1])], parts_old[0]]
    return "-".join(parts_new)

In [8]:
sample_combined_df.sample()

,Symbol,Series,Date,Prev Close,Open Price,High Price,Low Price,Last Price,Close Price,Average Price,Total Traded Quantity,Turnover,No. of Trades,Deliverable Qty,% Dly Qt to Traded Qty
462579,NEOGEN,EQ,02-Jan-2020,362.75,360.3,365.0,358.55,361.85,361.15,362.32,7405,2682962.25,384,4914,66.36


In [ ]:
def merge_export_func(data_dict):
    new_merged_data = []
    output_data_loc = data_dict['output_data_loc']
    dirname = data_dict['dirname']
    filename = data_dict['filename']
    symbol = data_dict['symbol']
    
    full_old_df = pd.read_csv(os.path.join(dirname,filename))
    subset_combined_df = sample_combined_df[sample_combined_df["Symbol"] == symbol]

    full_old_df['Date'] = full_old_df['Date'].apply(lambda x: convert_date_to_nse_format(x))

    for idx, row in full_old_df.iterrows():
        #['Date' 'Symbol' 'Series' 'Prev Close' 'Open' 'High' 'Low' 'Last' 'Close'
         #'VWAP' 'Volume' 'Turnover' 'Trades' 'Deliverable Volume' '%Deliverble']
        new_merged_data.append({
            "Symbol":row["Symbol"],
            "Series":row["Series"],
            "Date":row["Date"],
            "Prev Close":row["Prev Close"],
            "Open Price":row["Open"],
            "High Price":row["High"],
            "Low Price":row["Low"],
            "Last Price":row["Last"],
            "Close Price":row["Close"],
            "Average Price":row["VWAP"],
            "Total Traded Quantity":row["Volume"],
            "Turnover":row["Turnover"],
            "No. of Trades":row["Trades"],
            "Deliverable Qty":row["Deliverable Volume"],
            "% Dly Qt to Traded Qty":row["%Deliverble"],
        })


    for idx, row in subset_combined_df.iterrows():
        # NSE Website Columnnames
        #Symbol	Series	Date	Prev Close	Open Price	High Price	Low Price	Last Price	Close Price	VWAP	Total Traded
        # Quantity	Turnover
        # Rs.	No. of
        # Trades	Deliverable
        # Qty	% Dly Qt to
        # Traded Qty

        new_merged_data.append({
            "Symbol":row["Symbol"],
            "Series":row["Series"],
            "Date":row["Date"],
            "Prev Close":row["Prev Close"],
            "Open Price":row["Open Price"],
            "High Price":row["High Price"],
            "Low Price":row["Low Price"],
            "Last Price":row["Last Price"],
            "Close Price":row["Close Price"],
            "Average Price":row["Average Price"],
            "Total Traded Quantity":row["Total Traded Quantity"],
            "Turnover":row["Turnover"],
            "No. of Trades":row["No. of Trades"],
            "Deliverable Qty":row["Deliverable Qty"],
            "% Dly Qt to Traded Qty":row["% Dly Qt to Traded Qty"],
        })
    


    for dt in new_merged_data:
        dt['Timestamp'] = dateutil.parser.parse(dt['Date']).timestamp()

    new_merged_df = pd.DataFrame(new_merged_data)
    new_merged_df.drop_duplicates(subset=["Date"],keep="last")
    new_merged_df.to_csv(f"{output_data_loc}/{symbol}.csv",index=False)



output_data_loc = "input/all_hist_data"
if not os.path.exists(output_data_loc):
    os.makedirs(output_data_loc)


merge_worker_data = []
    
for dirname, subdir, filenames in os.walk("input\\kaggle daily NSE data\\SCRIPT"):
    for filename in filenames:
        symbol = filename.split(".")[0]
        merge_worker_data.append({
            "output_data_loc": output_data_loc,
            "dirname": dirname,
            "symbol": symbol,
            "filename": filename,
        })


bs_threadify(merge_worker_data, merge_export_func, num_threads=16)
        


        
        

while -true
while -true
while -true
while -true
while -true
while -true
while -true
while -true
while -true
while -true
while -true
while -true
while -true
while -true
while -true
while -true
while -true
while -true
while -true
while -true
while -true
while -true
while -true
while -true
while -true
while -true
while -true
while -true
while -true
while -true
while -true
while -true
while -true
while -true
while -true
while -true
